# Implementing HSTW-MLv0.2

In [2]:
import numpy as np
import pickle
import librosa
from scipy.spatial.distance import cdist
import python_speech_features
from matplotlib import pyplot as plt
from librosa.sequence import dtw
from matplotlib import gridspec
import time
import os
import pandas as pd
import glob
from tqdm import tqdm
from multiprocessing import Pool
from multiprocessing import cpu_count
from scipy.optimize import brentq
from scipy.interpolate import interp1d
from sklearn.metrics import roc_curve, auc
from offset import find_offset as offset_hps
from random import sample

## the HSTW portion of the system

Code is adapted from HSTW GitHub repo notebook: 02a_HPTWAlign.ipynb

In [3]:
%load_ext Cython

In [36]:
%%cython -a

import numpy as np
cimport numpy as np
cimport cython

DTYPE_INT32 = np.int32
ctypedef np.int32_t DTYPE_INT32_t

DTYPE_FLOAT = np.float64
ctypedef np.float64_t DTYPE_FLOAT_t

cdef DTYPE_FLOAT_t MAX_FLOAT = float('inf')

# @cython.boundscheck(False)
def NWTWDP(double[:,:]C, float alpha, int beta=20, int gamma=1):
    # 0: visible, 1: hidden
    # B: 1 Diag, 2 Right, 3 Up, 0 switch plane
    # initialization
    
    cdef DTYPE_INT32_t numRows = C.shape[0]
    cdef DTYPE_INT32_t numCols = C.shape[1]
    
    cdef np.ndarray[np.uint32_t, ndim=3] B = np.zeros((2, numRows,numCols), dtype=np.uint32)
    cdef np.ndarray[DTYPE_FLOAT_t, ndim=3] D = np.ones((2, numRows, numCols), dtype=DTYPE_FLOAT) * MAX_FLOAT

    cdef DTYPE_INT32_t i, j
    cdef unsigned int best_step
    cdef DTYPE_FLOAT_t cost, new_cost
    
    # bottom rows
    D[0, 0] = C[0]
    
    # first cols
    for i in range(1, C.shape[0]):
        D[1, i, 0] = D[1, i-1, 0] + alpha
        D[0, i, 0] = D[1, i, 0] + beta
        B[0, i, 0] = 3
        B[1, i, 0] = 0
        
    # rest of the matrix
    for i in range(1, C.shape[0]):
        for j in range(1, C.shape[1]):
        
            # hidden
            # diag visible -> hidden, right in hidden, up in hidden
            
            cost = D[0, i-1, j-1] + gamma + alpha
            step = 0
            new_cost = D[1, i, j-1] + gamma
            if new_cost < cost:
                cost = new_cost
                step = 2
            new_cost = D[1, i-1, j] + alpha
            if new_cost < cost:
                cost = new_cost
                step = 3
            D[1, i, j] = cost
            B[1, i, j] = step
             
            # visible
            # hidden -> visible, diag
            cost = D[1, i, j] + beta
            step = 0
            new_cost = D[0, i-1, j-1] + C[i, j]
            if new_cost < cost:
                cost = new_cost
                step = 1
            D[0, i, j] = cost
            B[0, i, j] = step
    return B, D

@cython.boundscheck(False)
def backtrace3D(unsigned int[:,:,:] B, double[:,:,:] D):
    cdef int p, r, c
    cdef unsigned int step = 0
    
    p = 0
    r = D.shape[1] - 1
    c = np.argmin(D[0, D.shape[1] - 1])
    cdef np.ndarray[np.int32_t, ndim=2] path_3D = np.zeros((D.shape[1]+D.shape[2], 3), dtype=np.int32)
    
    while r >= 0:
        path_3D[step] = [p,r,c]
        step += 1
        if B[p, r, c] == 0 and p == 0:
            p = 1
            r -= 1
            c -= 1
        elif B[p, r, c] == 0 and p == 1:
            p = 0
        elif B[p, r, c] == 1:
            r -= 1
            c -= 1
        elif B[p, r, c] == 2:
            c -= 1
        elif B[p, r, c] == 3:
            r -= 1
    return path_3D[:step]

In file included from /home/hlei/anaconda3/envs/mir/lib/python3.9/site-packages/numpy/core/include/numpy/ndarraytypes.h:1969,
                 from /home/hlei/anaconda3/envs/mir/lib/python3.9/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /home/hlei/anaconda3/envs/mir/lib/python3.9/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from /home/hlei/.cache/ipython/cython/_cython_magic_b64814e599820d0977b029bbf25bf797.c:715:
/home/hlei/anaconda3/envs/mir/lib/python3.9/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~


In [5]:
#Aligns a query file with its corresponding reference file and returns the 3-D path throught the HSTW tensor
def alignNWTWDP3D(C, Ca = 2.4, Cb = 33, gamma = 3):
    alpha = np.median(np.min(C, axis=1)) * Ca
    B, D = NWTWDP(C, alpha, beta=(alpha+gamma)*Cb)
    path_3D = backtrace3D(B, D)
    return path_3D

## the ML portion of the system

In [6]:
def find_offset(C):
    """
    C (np.array) : cost matrix of ref and query
    
    returns
        min_offset (int) : frame at optimal diagonal path
    """
    diag_sums = [C.diagonal(i).sum() for i in range(C.shape[1]-C.shape[0])]
    min_offset = np.argmin(diag_sums)  
    return min_offset

def find_matching_frames(offset, path, threshold=0):
    """
    offset (int) : frame at optimal diagonal path
    path (np.array) : 3 columns representing [hidden? (bool), query frame, ref frame]
    
    returns
        matching (np.array) : 3 columns representing [not matching? (bool), query frame, ref frame]
    """
    matching = path.copy()
    for idx, x in enumerate(matching):
        plane, q, r = x
        if plane == 0 and abs(r - q - offset) > threshold:
            matching[idx][0] = 1
    if np.sum(matching[:,0]) == 0:
        return None
    return matching

def calculate_scores_H1(query, ref, matching, return_all=False):
    """
    query (np.array) : mfcc features for query
    ref (np.array) : mfcc features for reference
    matching (np.array) : 3 columns representing [matching? (bool), query frame, ref frame]
    
    returns
        (float) : H1 modified z-score 
    """
    matching_diffs = np.array([query[q] - ref[r] for m, q, r in matching if m == 0])
    non_matching_diffs = np.array([query[q] - ref[r] for m, q, r in matching if m == 1])
    mean, std = matching_diffs.mean(axis=0), matching_diffs.std(axis=0)
    scores = (non_matching_diffs - mean) / std
    if return_all:
        return scores, mean, std
    return np.abs(scores).mean()

def calculate_scores_H2(query, ref, matching, return_all=False):
    """
    query (np.array) : mfcc features for query
    ref (np.array) : mfcc features for reference
    matching (np.array) : 3 columns representing [matching? (bool), query frame, ref frame]
    
    returns
        (float) : H2 modified z-score 
    """
    q_matching = query[matching[matching[:,0]==0][:,1]]
    r_matching = ref[matching[matching[:,0]==0][:,2]]
    mean, std = q_matching.mean(axis=0) - r_matching.mean(axis=0), q_matching.std(axis=0) + r_matching.std(axis=0)
    non_matching_diffs = np.array([query[q] - ref[r] for m, q, r in matching if m == 1])
    scores = (non_matching_diffs - mean) / std
    if return_all:
        return scores, mean, std
    return np.abs(scores).mean()

In [7]:
def sec_to_mfcc(sec):
    """
    sec (float): the optimal offset in seconds
    
    returns 
        (int): the corresponding mfcc frame
    """
    winstep = 0.01
    
    return round(sec/winstep)

## lets start the big loop

In [8]:
mfcc_DIR = './daps-mp3/test/mfccs/'
#HP_DIR = './daps-mp3/train/hashprints/'

queries = ['queries/' + file[:-4] for file in sorted(os.listdir(mfcc_DIR + 'queries/'))]
tamp_025 = ['tampered0.25/' + file[:-4] for file in sorted(os.listdir(mfcc_DIR + 'tampered0.25/'))]
tamp_05 = ['tampered0.5/' + file[:-4] for file in sorted(os.listdir(mfcc_DIR + 'tampered0.5/'))]
tamp_1 = ['tampered1/' + file[:-4] for file in sorted(os.listdir(mfcc_DIR + 'tampered1/'))]
tamp_2 = ['tampered2/' + file[:-4] for file in sorted(os.listdir(mfcc_DIR + 'tampered2/'))]
tamp_4 = ['tampered4/' + file[:-4] for file in sorted(os.listdir(mfcc_DIR + 'tampered4/'))]

In [9]:
#ref_hp_dict = {file[:-10]: np.load(HP_DIR + 'refs/'+file) for file in sorted(os.listdir(HP_DIR + 'refs/'))}
ref_mfcc_dict = {file[:-10]: np.load(mfcc_DIR + 'refs/'+file) for file in sorted(os.listdir(mfcc_DIR + 'refs/'))}

In [10]:
all_queries = queries + tamp_025 + tamp_05 + tamp_1 + tamp_2 + tamp_4

In [11]:
def calculate_tamper_score(query):
    query_type, query_name = query.split('/')

    if query_type == "queries":
        tamper_type = "NONE"
        tamper_len = 0.
    else:
        tamper_type = query_name[:3].upper()
        tamper_len = float(query_type[len('tampered'):])
    
    _, query_no, speaker, script, _ = query_name.split('_')
    _, bitrate = query_name.split('-')
    ref_name = f'{speaker}_{script}'
    
    # load query mfcc
    query_mfcc = np.load(mfcc_DIR + query + '.npy')
    
    # load ref mfcc
    ref_mfcc = ref_mfcc_dict[ref_name]
    
    # threshold delta delta and find offset
    query_mhps = np.dot(query_mfcc[:,13:] > 0,np.power(2,np.arange(26))[::-1]).tolist()
    ref_mhps = np.dot(ref_mfcc[:,13:] > 0,np.power(2,np.arange(26))[::-1]).tolist()
    
    offset = offset_hps(query_mhps, ref_mhps)
    
    start = max(0, offset-sec_to_mfcc(2.5))
    end = min(offset+query_mfcc.shape[0]+sec_to_mfcc(2.5), ref_mfcc.shape[0])

    
    ref_mfcc = ref_mfcc[start:end]
    
    C = cdist(query_mfcc, ref_mfcc).astype('float64')
    
    # use HSTW to align query to reference
    path = alignNWTWDP3D(C, Ca = 2.4, Cb = 33, gamma = 3)
    
    # find best diagonal path 
    best_offset = find_offset(C)
    
    # classify frames as matching or not
    match = find_matching_frames(best_offset, path, threshold=0)
    
    if match is None:
        tamper_score = 0
        h1 = 0
    elif np.all(match[:,0] == 1):
        tamper_score = h1 = 100
    else:
        h1 = calculate_scores_H1(query_mfcc, ref_mfcc, match, return_all=False)
        h2 = calculate_scores_H2(query_mfcc, ref_mfcc, match, return_all=False)
        
        tamper_score = h1 - h2
    
    return tamper_type, tamper_len, bitrate, ref_name, query_no, tamper_score

In [42]:
p = Pool(39)
with p:
    results_queries = list(tqdm(p.imap_unordered(calculate_tamper_score, all_queries), total=len(all_queries)))

100%|████████████████████████████████████| 24000/24000 [02:33<00:00, 156.02it/s]


In [35]:
df

,type,len,bitrate,ref,query_no,score
0,NONE,0.0,128k,f6_script1,0,0.000000
1,NONE,0.0,64k,f6_script1,0,0.000000
2,NONE,0.0,128k,f6_script2,0,0.000000
3,NONE,0.0,256k,f6_script3,0,0.000000
4,NONE,0.0,256k,f7_script2,0,0.000000
...,...,...,...,...,...,...
23995,REP,4.0,128k,m9_script3,9,2.150809
23996,REP,4.0,64k,m9_script4,9,2.624625
23997,REP,4.0,64k,m9_script5,9,3.968824
23998,REP,4.0,256k,m9_script5,9,4.208697


In [15]:
df = pd.DataFrame(columns=['type', 'len', 'bitrate', 'ref', 'query_no', 'score'], data=results_queries)

In [16]:
outdir = 'mfccs'
os.makedirs(f'./daps-mp3/results/{outdir}', exist_ok=True)
df.to_csv(f'./daps-mp3/results/{outdir}/HSTW_cython_test.csv')

In [17]:
def calculate_eer(fpr, tpr):
    '''
    requires fpr, tpr output from roc_curve (sklearn.metrics)
    Returns the equal error rate for a binary classifier output.
    '''
    eer = brentq(lambda x : 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
    
    return eer*100

In [18]:
labels = ['INS','DEL','REP','aggregate']
tamperlens = [4, 2, 1, 0.5, 0.25, 'aggregate']

In [19]:
def get_eer_table(results_df, bitrate):
    
    results_df = results_df[results_df["bitrate"] == bitrate]
    
    results_df.loc[results_df['type'] != 'NONE', 'truth'] = 1
    results_df.loc[results_df['type'] == 'NONE', 'truth'] = 0

    total = {'tamper_len':tamperlens, 'INS':[] ,'DEL':[] ,'REP':[] ,'aggregate':[]}
    for label in labels:
        cols = []
        if label != 'aggregate':
            lab = results_df[(results_df["type"] == label) | (results_df["type"] == 'NONE')]
        else:
            lab = results_df
        
        for lens in tamperlens:
            if lens != 'aggregate':
                len_lab = lab[(lab['len'] == lens) | (lab['len'] == 0)]
            else:
                len_lab = lab
            fpr, tpr, thresholds = roc_curve(len_lab['truth'], len_lab['score'])
            eer = calculate_eer(fpr, tpr)
            cols.append(eer)
            
        total[label]=cols
        
    df = results_df.astype(str)
    #df = df.style.set_caption('bitrate: '+ str(bitrate)).hide_index()
    df = pd.DataFrame(data=total)
    display(df) 
    
    return df

In [20]:
for bitrate in ['256k', '128k', '64k']:
    get_eer_table(df, bitrate)

/tmp/ipykernel_2743260/2183605032.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df.loc[results_df['type'] != 'NONE', 'truth'] = 1


,tamper_len,INS,DEL,REP,aggregate
0,4,0.000000,0.000000,0.000000,0.000000
1,2,0.000000,0.000000,0.000000,0.000000
2,1,0.000000,0.000000,0.000000,0.000000
3,0.5,0.199601,0.199601,3.100775,1.185771
4,0.25,0.199601,0.000000,33.065596,14.187643
5,aggregate,0.079936,0.039984,9.518639,3.425187


/tmp/ipykernel_2743260/2183605032.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df.loc[results_df['type'] != 'NONE', 'truth'] = 1


,tamper_len,INS,DEL,REP,aggregate
0,4,0.000000,0.000000,0.000000,0.000000
1,2,0.000000,0.000000,0.000000,0.000000
2,1,0.000000,0.000000,0.000000,0.000000
3,0.5,0.199601,0.398406,3.100775,1.250823
4,0.25,0.199601,0.000000,34.554974,15.014164
5,aggregate,0.079936,0.079936,10.071942,3.648510


/tmp/ipykernel_2743260/2183605032.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df.loc[results_df['type'] != 'NONE', 'truth'] = 1


,tamper_len,INS,DEL,REP,aggregate
0,4,0.000000,0.000000,0.000000,0.000000
1,2,0.000000,0.000000,0.000000,0.000000
2,1,0.000000,0.000000,0.199601,0.066622
3,0.5,0.199601,0.398406,4.030710,1.574803
4,0.25,0.199601,0.000000,38.423645,17.264203
5,aggregate,0.079936,0.079936,11.785462,4.312325
